In [41]:
import pandas as pd
import numpy as np
import requests
import time
from config import api_key
import pymongo

In [46]:
client = pymongo.MongoClient("mongodb://localhost:27017")

In [30]:
cities = pd.read_csv(r"static\resources\cities.csv")
cities.head()

,rank,name,usps,pop2021,pop2010,growth,density,aland_sqmi
0,1,New York City,NY,8230290,8190210,0.0049,27400,300.381
1,2,Los Angeles,CA,3983540,3795510,0.0495,8494,468.956
2,3,Chicago,IL,2679080,2697480,-0.0068,11783,227.369
3,4,Houston,TX,2323660,2100280,0.1064,3630,640.194
4,5,Phoenix,AZ,1733630,1449040,0.1964,3349,517.673


In [31]:
city_list = []
for city in cities['name']:
    city_list.append(city)
    
    

In [32]:
url = 'http://api.openweathermap.org/data/2.5/weather?'
units = 'metric'
query_url = f'{url}appid={api_key}&units={units}&q='

city_name = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []


for city in city_list:
        response = requests.get(query_url + city).json()
        city_name.append(city)
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        cloudiness.append(response['clouds']['all'])
        date.append(response['dt'])


In [33]:
weather = {'City':city_name,'Lat':lat,'Lng':lng,'Max Temp':max_temp,'Humidity':humidity,'Cloudiness':cloudiness,'Wind Speed':wind_speed,'Country':country,'Date':date}
weather_df = pd.DataFrame({ key:pd.Series(value) for key, value in weather.items() })



In [35]:
weather_df.to_csv(r'static\resources\weather_df.csv', index=False)

In [44]:
weather_data = weather_df.to_dict(orient='records')


In [49]:
db = client["city_weather_db"]
db.weather.insert_many(weather_data)

In [36]:
url = 'http://api.openweathermap.org/data/2.5/forecast?'
units = 'metric'
query_url = f'{url}appid={api_key}&units={units}&q='


city_name = []
lat_5 = []
lng_5 = []
max_temp_5 = []
humidity_5 = []
cloudiness_5 = []
wind_speed_5 = []
date_5 = []


for city in city_list:
        for i in range(0,40,8):
                response = requests.get(query_url + city).json()
                city_name.append(city)
                lat_5.append(response['city']['coord']['lat'])
                lng_5.append(response['city']['coord']['lon'])
                max_temp_5.append(response['list'][i]['main']['temp_max'])
                humidity_5.append(response['list'][i]['main']['humidity'])
                wind_speed_5.append(response['list'][i]['wind']['speed'])
                cloudiness_5.append(response['list'][i]['clouds']['all'])
                date_5.append(response['list'][i]['dt_txt'])


In [37]:
weather_5 = {'City':city_name,'Lat':lat_5,'Lng':lng_5,'Max Temp':max_temp_5,'Humidity':humidity_5,'Cloudiness':cloudiness_5,'Wind Speed':wind_speed_5,'Date':date_5}
weather5_df = pd.DataFrame({ key:pd.Series(value) for key, value in weather_5.items() })



In [38]:
weather5_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,New York City,40.7143,-74.0060,11.45,54,65,4.44,2021-11-17 18:00:00
1,New York City,40.7143,-74.0060,17.38,51,69,5.43,2021-11-18 18:00:00
2,New York City,40.7143,-74.0060,8.02,37,31,7.37,2021-11-19 18:00:00
3,New York City,40.7143,-74.0060,7.42,38,64,2.19,2021-11-20 18:00:00
4,New York City,40.7143,-74.0060,11.96,60,99,3.54,2021-11-21 18:00:00
5,Los Angeles,34.0522,-118.2437,18.90,76,78,1.30,2021-11-17 18:00:00
6,Los Angeles,34.0522,-118.2437,20.00,49,78,1.59,2021-11-18 18:00:00
7,Los Angeles,34.0522,-118.2437,19.16,53,100,1.07,2021-11-19 18:00:00
8,Los Angeles,34.0522,-118.2437,18.95,50,99,1.23,2021-11-20 18:00:00
9,Los Angeles,34.0522,-118.2437,22.39,17,100,1.17,2021-11-21 18:00:00


In [40]:
weather5_df.to_csv(r'static\resources\weather_5.csv', index=False)

In [50]:
data5 = weather5_df.to_dict(orient='records')

In [51]:
db=client["city_weather_db"]
db.weather_5.insert_many(data5)